In [1]:
import wave
import numpy as np

# Open the .wav file
# with wave.open("../demo/tts-output-1.wav", 'rb') as wav_file:
with wave.open("../demo/warning.wav", 'rb') as wav_file:
    # Extract audio parameters
    num_channels = wav_file.getnchannels()
    sample_width = wav_file.getsampwidth()
    frame_rate = wav_file.getframerate()
    num_frames = wav_file.getnframes()
    # Read audio data
    audio_data = wav_file.readframes(num_frames)
    print(num_channels,sample_width,frame_rate)
    
# np_array=np.frombuffer(audio_data,dtype=np.int16)

2 2 48000


In [50]:
sample_rate

22050

In [2]:
import torch, torchaudio

np_array=np.frombuffer(audio_data,dtype=np.int16)

np_array = np_array.reshape(-1, 2).mean(axis = 1)
# np_array=np_array.astype(np.float32) /32768.0
# audio_tensor = torch.from_numpy(np_array)
# audio_data = torchaudio.functional.resample(audio_tensor, 22050, 48000)
# # Step 4: Convert mono tensor to stereo by duplicating the channel
# stereo_tensor = torch.stack([audio_data, audio_data], dim=0)

# np_stereo=stereo_tensor.numpy()

In [4]:
# speech_unsq = torch.unsqueeze(stereo_tensor, dim=0)
# torchaudio.save("warning1.wav", audio_tensor, 48000)
# Open a .wav file in write mode
with wave.open('warning1.wav', 'wb') as wf:
    # Set the parameters
    n_channels = 1
    sampwidth = 2  # Number of bytes per sample (16-bit audio is 2 bytes)
    wf.setnchannels(n_channels)
    wf.setsampwidth(sampwidth)
    wf.setframerate(48000)

    # Write the audio data to the file
    wf.writeframes(np_array.tobytes())

In [31]:
audio.dtype

dtype('float64')

In [1]:
my_dictionary={}

In [2]:
chunk=b''

In [3]:
my_dictionary[chunk]='yes'

In [4]:
chunk2=b'chunk2'
my_dictionary[chunk2]='yes'

In [5]:
my_dictionary

{b'': 'yes', b'chunk2': 'yes'}

In [8]:
list(my_dictionary.values())

['yes', 'yes']

# Record Audio

In [5]:
import pyaudio

CHUNK = 1024
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 48000
RECORD_SECONDS = 5

# audio_buffer=b''
audio_buffer=[]

p = pyaudio.PyAudio()
stream = p.open(format=FORMAT,
                channels=CHANNELS,
                rate=RATE,
                input=True,
                frames_per_buffer=CHUNK)

print("* recording")

for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
    data = stream.read(CHUNK)
    # audio_buffer += data
    audio_buffer.append(data)

print("* done recording")

stream.stop_stream()
stream.close()
p.terminate()

* recording
* done recording


In [6]:
import wave

output_audio_file='../demo/warning.wav'

# Join Entire data from audio_buffer
audio_data_bytes = b''.join(audio_buffer)

with wave.open(output_audio_file, "wb") as audio_file:
    n_channels = 1 # Number of channels
    sampwidth = 2  # Sample width in bytes (e.g., 2 bytes for 16-bit audio)
    framerate = 48000  # Frame rate (samples per second)
    n_frames = len(audio_data_bytes) // sampwidth

    audio_file.setnchannels(n_channels)
    audio_file.setsampwidth(sampwidth)
    audio_file.setframerate(framerate)
    audio_file.writeframes(audio_data_bytes)

In [17]:
io.BytesIO(audio_data_bytes).seek(0)

0

In [22]:
temp=io.BytesIO(audio_data_bytes).seek(0,io.SEEK_END)
temp

1320960

In [1]:
import wave
import numpy as np

# Parameters for the silent audio file
duration_seconds = 120  # 2 minutes
sample_rate = 44100
num_channels = 2
sample_width = 2

# Calculate the number of frames
num_frames = duration_seconds * sample_rate

# Create silent audio data
silent_audio = np.zeros((num_frames, num_channels), dtype=np.int16)

# Convert the silent audio data to bytes
silent_audio_bytes = silent_audio.tobytes()

# Create a wave file and write the silent audio data to it
filename = "silent_audio.wav"
with wave.open(filename, 'wb') as wf:
    wf.setnchannels(num_channels)
    wf.setsampwidth(sample_width)
    wf.setframerate(sample_rate)
    wf.writeframes(silent_audio_bytes)

# VAD test

In [54]:
import torch, torchaudio

prob_data=[]

# loading model for vad
model, utils = torch.hub.load(repo_or_dir='snakers4/silero-vad',
                              model='silero_vad',
                              force_reload=True,
                              onnx=False)

# Global constants related to the audio input format and chosen chunk values.
# Edit as appropriate for the input stream.
SAMPLE_RATE = 16000
ORIG_SAMPLE = 44100
SILENCE_TIME = 2 # seconds
CHUNK_SAMPLES = 512
CHANNELS = 1
BIT_DEPTH = 2
CHUNK_SIZE = int(CHUNK_SAMPLES * CHANNELS * BIT_DEPTH * (ORIG_SAMPLE/ SAMPLE_RATE)) # amt of bytes per chunk
SILENCE_SAMPLES = SAMPLE_RATE * SILENCE_TIME

resample = torchaudio.transforms.Resample(orig_freq = ORIG_SAMPLE, new_freq = SAMPLE_RATE)

speech_threshold=0.0

test=b''

def VAD(chunk, threshold_weight=0.8):
    global speech_threshold, test

    np_chunk = np.frombuffer(chunk, dtype = np.int16)
    np_chunk = np_chunk.astype(np.float32) / 32768.0
    # np_chunk = np_chunk.reshape(-1, CHANNELS).mean(axis = 1)
    chunk_audio = torch.from_numpy(np_chunk)
    chunk_audio = resample(chunk_audio)

    # Find prob of speech for using silero-vad model
    speech_prob = model(chunk_audio, SAMPLE_RATE).item()
    prob_data.append(speech_prob)

    if speech_prob>speech_threshold:
        # print('speech ', speech_threshold)
        test+=chunk

    # Adaptive thresholding which should allow for silence at the beginning
    # of audio and adapt to differing confidence levels of the VAD model.
    # Equation acquired from link:
    # https://vocal.com/voice-quality-enhancement/voice-activity-detection-with-adaptive-thresholding/
    speech_threshold = threshold_weight * max([i**2 for i in prob_data]) + (1 - threshold_weight) * min([i**2 for i in prob_data])
    print(speech_threshold)

Downloading: "https://github.com/snakers4/silero-vad/zipball/master" to C:\Users\shtab/.cache\torch\hub\master.zip


In [55]:
import io, numpy as np

offset=0

# audio_data_bytes=io.BytesIO(audio_data_bytes)
size=audio_data_bytes.seek(0, io.SEEK_END) # seek to end of audio
 
while offset < size:
    audio_data_bytes.seek(offset)
    chunk = audio_data_bytes.read(CHUNK_SIZE)
    VAD(chunk)
    offset += CHUNK_SIZE


0.00015236065403964952
0.0001329422440397535
0.0001270405817374112
0.00012637604730167471
0.0001262058010250936
0.0001249744494620621
0.00012389191827200704
0.00012327076401835518
0.00012307345962725847
0.00012298881392862066
0.00012290629718246396
0.0001228670764628631
0.0001228670764628631
0.0001228670764628631
0.0001228670764628631
0.0001228670764628631
0.009124395282111473
0.009124395282111473
0.009124395282111473
0.009124395282111473
0.02268813692102237
0.44177832787980537
0.5729160184516955
0.6367531465732723
0.6367531465732723
0.6367531465732723
0.6367531465732723
0.6367531465732723
0.6367531465732723
0.6367531465732723
0.6367531465732723
0.6367531465732723
0.6367531465732723
0.6367531465732723
0.6367531465732723
0.6367531465732723
0.6367531465732723
0.6367531465732723
0.6367531465732723
0.6367531465732723
0.6367531465732723
0.6367531465732723
0.6367531465732723
0.6367531465732723
0.6367531465732723
0.6367531465732723
0.6367531465732723
0.6367531465732723
0.6367531465732723
0.63

Error: The following operation failed in the TorchScript interpreter.
Traceback of TorchScript, serialized code (most recent call last):
  File "code/__torch__/vad/model/vad_annotator.py", line 123, in forward
    _21 = torch.gt(torch.div(sr1, (torch.size(x3))[1]), 31.25)
    if _21:
      ops.prim.RaiseException("Input audio chunk is too short", "builtins.ValueError")
      ~~~~~~~~~~~~~~~~~~~~~~~ <--- HERE
    else:
      pass

Traceback of TorchScript, original code (most recent call last):
  File "/home/keras/notebook/nvme1/adamnsandle/silero-models-research/vad/model/vad_annotator.py", line 472, in forward
    
        if sr / x.shape[1] > 31.25:
            raise ValueError("Input audio chunk is too short")
            ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ <--- HERE
    
        return x, sr
builtins.ValueError: Input audio chunk is too short


In [47]:
with wave.open('vadoutput.wav', "wb") as audio_file:
    n_channels = 1 # Number of channels
    sampwidth = 2  # Sample width in bytes (e.g., 2 bytes for 16-bit audio)
    framerate = 44100  # Frame rate (samples per second)
    n_frames = len(test) // sampwidth

    audio_file.setnchannels(n_channels)
    audio_file.setsampwidth(sampwidth)
    audio_file.setframerate(framerate)
    audio_file.writeframes(test)

In [41]:
prob_data

[0.012343445792794228,
 0.007541054394096136,
 0.005013922695070505,
 0.00466985022649169,
 0.004761974792927504,
 0.003973800223320723,
 0.003158211475238204,
 0.0026695390697568655,
 0.002450003055855632,
 0.002321795793250203,
 0.002246081130579114,
 0.0022464424837380648,
 0.002268302720040083,
 0.0023725901264697313,
 0.0024567015934735537,
 0.0024250445421785116,
 0.0024908219929784536,
 0.10768572986125946,
 0.10175097733736038,
 0.04164091497659683,
 0.05254501849412918,
 0.16853904724121094,
 0.7435227036476135,
 0.8468866348266602,
 0.8919724225997925,
 0.8284968137741089,
 0.7942655086517334,
 0.7572094202041626,
 0.6633505821228027,
 0.4709952473640442,
 0.41446274518966675,
 0.44212713837623596,
 0.43384480476379395,
 0.5142945051193237,
 0.5822402834892273,
 0.477008581161499,
 0.4326702058315277,
 0.30586037039756775,
 0.1896154135465622,
 0.21422721445560455,
 0.1152985543012619,
 0.060878753662109375,
 0.03986699506640434,
 0.030567485839128494,
 0.13693010807037354,
 

(0.9967741797039622, 7.78461139258318e-09)

In [38]:
audio_buffer_copy=audio_buffer.copy()

In [366]:
audio_buffer=audio_buffer_copy.copy()

In [23]:
# import wave

# output_audio_file='1.wav'

# # Join Entire data from audio_buffer
# audio_data_bytes = b''.join(audio_buffer)

# with wave.open(output_audio_file, "wb") as audio_file:
#     n_channels = 1 # Number of channels
#     sampwidth = 2  # Sample width in bytes (e.g., 2 bytes for 16-bit audio)
#     framerate = 44100  # Frame rate (samples per second)
#     n_frames = len(audio_data_bytes) // sampwidth

#     audio_file.setnchannels(n_channels)
#     audio_file.setsampwidth(sampwidth)
#     audio_file.setframerate(framerate)
#     audio_file.writeframes(audio_data_bytes)

In [4]:
len(audio_data_bytes)

880640

# Testing VADIterator

In [55]:
from silero_vad import SileroVAD, VADIterator

# Initialize SileroVAD
onnx_model = './silero_vad.onnx'
vad = SileroVAD(onnx_model=onnx_model)

In [56]:
vad_iterator = VADIterator()

In [57]:
import numpy as np

audio_data = np.frombuffer(audio_data_bytes, dtype=np.int16)

for timestamp, frame in vad_iterator(audio_data, use_energy=False, return_seconds=False):
    if timestamp:
        print(timestamp, frame)

{'start': 20416} []
{'end': 24640} []
{'start': 29632} []
{'end': 48704} []
{'start': 65472} []
{'end': 85568} []
{'start': 140224} []
{'end': 158272} []
{'start': 408000} []
{'end': 424512} [-286. -328. -318. -265. -195. -139. -118. -135. -187. -251. -300. -314.
 -292. -254. -223. -219. -233. -251. -254. -235. -201. -165. -135. -108.
  -88.  -77.  -83. -107. -139. -163. -160. -132.  -92.  -65.  -62.  -78.
  -96.  -97.  -80.  -52.  -27.   -7.   -1.   -8.  -26.  -57.  -93. -118.
 -123. -110.  -86.  -64.  -57.  -63.  -79.  -91.  -88.  -60.  -11.   51.
  104.  131.  123.   92.   62.   60.   97.  160.  220.  250.  242.  214.
  199.  213.  255.  300.  326.  319.  290.  254.  227.  211.  201.  188.
  162.  124.   77.   34.    3.   -7.   -1.   16.   27.   22.    4.  -10.
   -3.   32.   79.  117.  133.  137.  156.  207.  286.  369.  429.  454.
  462.  475.  505.  551.  598.  626.  621.  581.  522.  459.  410.  376.
  350.  317.  267.  206.  145.  100.   76.   67.   62.   53.   40.   38.
   52.

# Combined Message Chunks

In [369]:
from silero_vad import SileroVAD

combined_message_chunks=[]

# Initialize SileroVAD
onnx_model = './silero_vad.onnx'
vad = SileroVAD(onnx_model=onnx_model)

In [370]:
combined_message_chunks

[]

# Main Logic using SileroVAD

In [371]:
import numpy as np
import soundfile as sf
import numpy as np
import tempfile
import os

# buffer to store combined 20 audio-chunks as one chunk
buffer=[]

# buffer index
i=0

# dictionary to store if buffer index has speech
hasSpeech={}

while len(audio_buffer)!=0:
    if len(combined_message_chunks) < 20:
        combined_message_chunks.append(audio_buffer.pop(0))
        
    if len(combined_message_chunks) ==20:
        
        combined_message = b''.join(combined_message_chunks)
        buffer.append(combined_message)

        combined_message_chunks.clear()
        audio_array=np.frombuffer(combined_message, dtype=np.int16)

        # Create tmppath as vad expects wav path 
        with tempfile.NamedTemporaryFile(suffix='.wav', delete=False) as tmpfile:
            sf.write(tmpfile.name,audio_array, RATE)
            tmpfile_path = tmpfile.name

        speech_timestamps = vad.get_speech_timestamps(tmpfile_path)
        hasSpeech[i] = any(speech_timestamps)
        # for speech in speech_timestamps:
        #     if speech:
        #         print(i,'contains speech', speech)

        # increate buffer index
        i=i+1

# \x02\x00

In [372]:
hasSpeech

{0: False,
 1: True,
 2: True,
 3: True,
 4: True,
 5: False,
 6: False,
 7: True,
 8: True,
 9: False,
 10: True,
 11: False,
 12: False,
 13: False,
 14: False,
 15: False,
 16: True,
 17: True,
 18: False,
 19: False,
 20: False}

# Reconstruct Audio

In [389]:
true_indexes = [k for k, v in hasSpeech.items() if v]
speech_segments=[]

for i in true_indexes:
    audio_array = np.frombuffer(buffer[i], dtype=np.int16)
    speech_segments.append(audio_array)

sf.write('fullaudio.wav',np.concatenate(speech_segments),44100)

In [319]:
len(buffer)

21

----------------------------------------------

In [118]:
import numpy as np

# Assuming your byte string is stored in audio_buffer
np.frombuffer(audio_data_bytes, dtype=np.int16)

array([ 0,  0, -1, ...,  4,  3,  3], dtype=int16)

In [119]:
len(np.frombuffer(audio_data_bytes, dtype=np.int16))

212992

# Check if speech is there in each chunk

In [51]:
from silero_vad import SileroVAD
import soundfile as sf
import numpy as np
import tempfile
import os

# Hardcode RATE
RATE=44100

# Initialize SileroVAD
onnx_model = './silero_vad.onnx'
vad = SileroVAD(onnx_model=onnx_model)

speech_segments = []

# Total number of samples processed so far
total_samples = 0

for i in range(len(audio_buffer)):

    # audio_array = np.frombuffer(audio_buffer[i], dtype=np.int16)
    audio_array = np.frombuffer(audio_data_bytes, dtype=np.int16)

    # Create a temporary WAV file to store the audio data
    with tempfile.NamedTemporaryFile(suffix='.wav', delete=False) as tmpfile:
        sf.write(tmpfile.name,audio_array, RATE)
        tmpfile_path = tmpfile.name

    # Get speech timestamps for the temporary WAV file
    speech_timestamps = vad.get_speech_timestamps(tmpfile_path)

    for speech in speech_timestamps:
        # Adjust the timestamps relative to the entire audio stream
        # start = speech['start'] 
        # end = speech['end'] 
        # print('chunk', i, 'contains speech', {'start': start, 'end': end})
        # speech_segments.append(audio_array[speech['start']:speech['end']])
        print(speech)
    total_samples += len(audio_array)

    # Clean up the temporary file
    os.unlink(tmpfile_path)

{'start': 29632} []
{'end': 48704} []
{'start': 65472} []

{'start': 25489, 'end': 73294}
{'start': 94638, 'end': 119863}
{'start': 25489, 'end': 73294}
{'start': 94638, 'end': 119863}
{'start': 25489, 'end': 73294}
{'start': 94638, 'end': 119863}
{'start': 25489, 'end': 73294}
{'start': 94638, 'end': 119863}
{'start': 25489, 'end': 73294}
{'start': 94638, 'end': 119863}
{'start': 25489, 'end': 73294}
{'start': 94638, 'end': 119863}
{'start': 25489, 'end': 73294}
{'start': 94638, 'end': 119863}
{'start': 25489, 'end': 73294}
{'start': 94638, 'end': 119863}
{'start': 25489, 'end': 73294}
{'start': 94638, 'end': 119863}
{'start': 25489, 'end': 73294}
{'start': 94638, 'end': 119863}
{'start': 25489, 'end': 73294}
{'start': 94638, 'end': 119863}
{'start': 25489, 'end': 73294}
{'start': 94638, 'end': 119863}
{'start': 25489, 'end': 73294}
{'start': 94638, 'end': 119863}
{'start': 25489, 'end': 73294}
{'start': 94638, 'end': 119863}
{'start': 25489, 'end': 73294}
{'start': 94638, 'end': 119863}
{'start': 25489, 'end': 73294}
{'start': 94638, 'end': 

KeyboardInterrupt: 

In [201]:
speech_segments

[]

In [199]:
sf.write('test.wav',np.concatenate(speech_segments),44100)

ValueError: need at least one array to concatenate

# Apply VAD to entire audio array

In [197]:
# Initialize SileroVAD
onnx_model = '../using-fastapi/silero-vad-basics/silero_vad.onnx'
vad = SileroVAD(onnx_model=onnx_model)

speech_segments = []

with tempfile.NamedTemporaryFile(suffix='.wav', delete=False) as tmpfile:
    audio_array = np.frombuffer(audio_data_bytes, dtype=np.int16)
    sf.write(tmpfile.name,audio_array, RATE)
    tmpfile_path = tmpfile.name

# Get speech timestamps for the temporary WAV file
speech_timestamps = vad.get_speech_timestamps(tmpfile_path)
for speech in speech_timestamps:
    print('contains speech', speech)
    speech_segments.append(audio_array[speech['start']:speech['end']])

sf.write('test.wav',np.concatenate(speech_segments),44100)

contains speech {'start': 55125, 'end': 74705}
contains speech {'start': 91816, 'end': 115630}
contains speech {'start': 138385, 'end': 159377}
contains speech {'start': 183544, 'end': 205947}
contains speech {'start': 232936, 'end': 255339}
contains speech {'start': 276683, 'end': 301908}
contains speech {'start': 330309, 'end': 355534}
contains speech {'start': 417803, 'end': 438272}


In [1]:
class MyClass:
    instances = []

    def __init__(self, name):
        self.name = name
        MyClass.instances.append(self)

# Creating instances of MyClass
obj1 = MyClass("Object 1")
obj2 = MyClass("Object 2")
obj3 = MyClass("Object 3")


In [5]:
del MyClass.instances[0]

In [6]:
MyClass.instances

[<__main__.MyClass at 0x1ac94b42640>, <__main__.MyClass at 0x1ac94b42460>]

In [25]:
# import garbage collection module
import gc

# # Create a list with a cyclic reference
# my_list = [1,2,3,4,5,6,7,8]
# my_list.append(my_list)

# # Delete the list
# del my_list

# Manually trigger garbage collection
collected1 = gc.collect()

# Verify memory release
print(f"Garbage collector collected {collected1} objects.")

Garbage collector collected 16 objects.


# Dictionary VAD

In [5]:
vad_dictionary={
    'chunk1':1,
    'chunk2':0,
    'chunk3':0,
    'chunk4':0,
    'chunk5':0,
    'chunk6':0,
    'chunk7':0
    }

all(i==0 for i in list(vad_dictionary.values())[- 48000*1:])

False

In [8]:
list(vad_dictionary.values())[- 48000 * 1:]

[1, 0, 0, 0, 0, 0, 0]

# Client Dictionary


In [1]:
from datetime import datetime

class Client:
    def __init__(self,client_id):
        self.client_id = client_id
        self.logs={}

    def addLogs(self):
        self.logs['audio.wav']=datetime.now().strftime("%Y-%m-%d %H:%M:%S")


In [2]:
clients={}
client1=Client(1)
clients['client1']=client1

client2=Client(2)
clients['client2']=client2

In [8]:
client2.addLogs()

In [3]:
clients

{'client1': <__main__.Client at 0x226f9c21160>,
 'client2': <__main__.Client at 0x226f9c21c40>}

In [5]:
list(clients.keys())

['client1', 'client2']

In [11]:
[{ client.client_id : client.logs}  for client in list(clients.values())]

[{1: {'audio.wav': '2024-08-06 16:20:02'}},
 {2: {'audio.wav': '2024-08-06 16:20:09'}}]

# Play numpy array

In [6]:
import pyaudio
import wave

# Path to the wave file
wave_file = '../demo/5sec_silence.wav'

# Open the wave file
wf = wave.open(wave_file, 'rb')

# Create an interface to PortAudio
p = pyaudio.PyAudio()

# Open a .Stream object to play the wave file
# 'output=True' indicates that the sound will be played rather than recorded
stream = p.open(format=p.get_format_from_width(wf.getsampwidth()),
                channels=wf.getnchannels(),
                rate=wf.getframerate(),
                output=True)

# Read data in chunks
chunk = 1024
data = wf.readframes(chunk)

# Play the audio by writing the audio data to the stream
while len(data) > 0:
    stream.write(data)
    data = wf.readframes(chunk)

# Stop and close the stream
stream.stop_stream()
stream.close()

# Close PyAudio
p.terminate()
